In [2]:
!pip install neo4j


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from neo4j import GraphDatabase

In [4]:
host = "bolt://localhost:7687"      # Connect to Neo4j 
auth = ("neo4j", "password")        # User and password

# Connect to the Neo4j Database Server
driver = GraphDatabase.driver(host, auth=auth)

# Create a Session 
session = driver.session()

# Check connection
session.run("MATCH (n) RETURN n.id LIMIT 1").data()

[{'n.id': '9e2501b8e5da6e1627508c4eb321bf66eb41020e'}]

In [5]:
# Create Nodes and Relationships
creation = '''
LOAD CSV WITH HEADERS FROM 'https://github.com/noachuartzt/IBD_Grupo2-P2/raw/main/1-publications/csv/output.csv' AS row

// Create Paper node
MERGE (p:Paper {id: row.paperId, title: row.title})
SET p.publicationDate = CASE WHEN row.publicationDate IS NOT NULL THEN row.publicationDate ELSE "Unknown" END
SET p.abstract = CASE WHEN row.abstract IS NOT NULL THEN row.abstract ELSE "Unknown" END

// Create Author node
WITH row, split(row.authorId, ',') AS ids, split(row.authorName, ',') AS names, p
UNWIND range(0, size(ids)-1) AS i

MERGE (a:Author {id: ids[i]})
SET a.name = names[i]

// Create WRITTEN_BY relationship
MERGE (p)-[:WRITTEN_BY {authorId: row.authorId}]->(a)
'''

result = session.run(creation)

In [6]:
collaborators = '''
MATCH (a:Author {name: "Y. Filali"})<-[:WRITTEN_BY]-(p:Paper)-[:WRITTEN_BY]->(b:Author) WHERE a <> b
WITH b, COUNT(p) AS Collaborations
ORDER BY Collaborations DESC
RETURN b.name AS Name, Collaborations 
'''

result2 = session.run(collaborators)

print("\n| Name | Collaborations |\n")

for record in result2:
    name, collaborations = record["Name"],  record["Collaborations"]
    print(f'| {name:<20} | {collaborations}')



| Name | Collaborations |

| A. Aarab             | 2
| M. A. Sabri          | 2
| O. Ukkola            | 1
| Y. Kesäniemi         | 1
| Humam Al-Shahwani    | 1
| Karan Aggarwal       | 1
| Dhamyaa Salim Mutar  | 1
| Maad M. Mijwil       | 1
| Hasnae El Khoukhi    | 1
| A. Yahyaouy          | 1
| A. Ennouni           | 1


In [7]:
articles1 = '''
MATCH (p:Paper)-[:WRITTEN_BY]->(:Author {name: 'Y. Filali'})
MATCH (p)-[:WRITTEN_BY]->(a:Author)
RETURN p.title AS Title, COUNT(a) AS numAuthors
ORDER BY numAuthors ASC
'''
result = session.run(articles1)

print("| Title | Number of authors |\n")

for record in result:
    num_authors, title = record["numAuthors"],  record["Title"]
    print(f'| {num_authors:<5} | {title}')

| Title | Number of authors |

| 3     | Soluble ST2, a biomarker of fibrosis, is associated with multiple risk factors, chronic diseases and total mortality in the OPERA study
| 4     | Skin Cancer Diagnosis Using an Improved Ensemble Machine Learning model
| 5     | Comparison between Expert Systems, Machine Learning, and Big Data: An Overview
| 5     | 2. An overview of skin lesion segmentation, features engineering, and classification


In [8]:
articles2 = '''
// Otra Opción
MATCH (a:Author {name: "Y. Filali"})<-[:WRITTEN_BY]-(p:Paper)-[:WRITTEN_BY]->(b:Author) WHERE a <> b
RETURN p.title as Title, COUNT (b) + 1 as numAuthors
ORDER BY numAuthors ASC
'''

result = session.run(articles2)

print("| Title | Number of authors |\n")

for record in result:
    num_authors, title = record["numAuthors"],  record["Title"]
    print(f'| {num_authors:<5} | {title}')

| Title | Number of authors |

| 3     | Soluble ST2, a biomarker of fibrosis, is associated with multiple risk factors, chronic diseases and total mortality in the OPERA study
| 4     | Skin Cancer Diagnosis Using an Improved Ensemble Machine Learning model
| 5     | Comparison between Expert Systems, Machine Learning, and Big Data: An Overview
| 5     | 2. An overview of skin lesion segmentation, features engineering, and classification
